# TODO:

* Check if file is actually a tar source
* 

# Obtain arxiv ids via RSS

In [1]:
import feedparser

import requests, shutil

In [2]:
feed = feedparser.parse('http://arxiv.org/rss/cond-mat.str-el')

In [28]:
ids = [entry['id'].rsplit('/',1)[-1] 
       for entry in feed['entries']]

[u'1712.04471',
 u'1712.04511',
 u'1712.04554',
 u'1712.04579',
 u'1712.04632',
 u'1712.04656',
 u'1712.04670',
 u'1712.04679',
 u'1712.04734',
 u'1712.04764',
 u'1712.04822',
 u'1712.04840',
 u'1712.04906',
 u'1712.04920',
 u'1712.04933',
 u'1703.02934',
 u'1706.05469',
 u'1708.07798',
 u'1708.08050',
 u'1709.07990',
 u'1710.09835',
 u'1712.01370']

# Download source file

In [ ]:
# Source file can be downloaded via
#https://arxiv.org/e-print/1712.04906

In [31]:
def id_to_url(id):
    "URL to download the source file for a paper"
    return "http://arxiv.org/e-print/" + id

In [ ]:
def download_source(id):
    url = id_to_url(id)
    r = requests.get(url, stream=True)
    
    filename = 'data/' + id + ".gz"

    with open(filename, 'wb') as f:
        f.write(r.content)
    
    del r

## Extract tex files

In [3]:
import tarfile

tar = tarfile.open("data/tmp.tar.gz")
tar.extractall("data/")
tar.close()

In [9]:
import os

files = os.listdir('data/.')
latex_files = [fn for fn in files if fn.endswith(".tex")]

In [10]:
latex_files

['main.tex', 'rixs.tex', 'sm.tex']

## Find comments in tex files

In [24]:
comments = []

with open('data/rixs.tex') as f:
    for line in f:
        if(line[0] == '%'):
            if(line[1] == '\'):
               continue
            else:
                comments.append(line)

SyntaxError: EOL while scanning string literal (<ipython-input-24-32621f5f2815>, line 6)

In [23]:
comments

['%\\documentclass[reprint,twocolumn,superscriptaddress,secnumarabic,amssymb, nobibnotes, aps, prb]{revtex4-1}\n',
 '%\\usepackage{amsmath}    % need for subequations\n',
 '%\\usepackage{graphicx}    % need for figures\n',
 '%\\usepackage{verbatim}    % useful for program listings\n',
 '%\\usepackage{color}           % use if color is used in text\n',
 '%\\usepackage{subfigure}   % use for side-by-side figures\n',
 '%\\usepackage{hyperref}    % use for hypertext links, including those to external documents \n',
 '%\\usepackage{verbatim}  % and URLs\n',
 '%\\usepackage{epstopdf}\n',
 "%\\raggedbottom                 % don't add extra vertical space\n",
 '%\\begin{comment}\n',
 '%\\pagestyle{empty}       % use if page numbers not wanted\n',
 '%\\end{comment}\n',
 '%\\renewcommand{\\thesubsection}{\\Alph{subsection}}\n',
 '%\\def\\laco*{LaCoO$_3$}\n',
 '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%\n',
 '%The low-energy physics of correlated insulators is typically \n',
 '%governe